## Q 1.1

In [1]:
from bs4 import BeautifulSoup
import requests
import re



c = 0
reg = r'href=\"https://[\w\W]*" id'
htt = list()
for i in range(8):
    cnt = requests.get("https://myanimelist.net/topanime.php?limit="+str(i*50))
    soup = BeautifulSoup(cnt.content, features="lxml")
    links = soup.find_all('h3')
    for l in range(len(links)-3):
        li = links[l].find('a')
        m = re.findall(reg, str(li))
        s = m[0][6:][:-4]+'\n'
        htt.append(s)
f = open("https.txt", 'w')
for i in htt:
    f.write(i)
f.close

<function TextIOWrapper.close()>

## Q 1.2

In [ ]:
with open('https.txt', 'r',encoding='utf-8') as f:
    for line in enumerate(f): 
            req = requests.get(line[1])
            type(req)
            with open("test.html", 'w') as file:
                file.write(req.text)
                